In [43]:
# https://www.youtube.com/watch?v=udeQhZHx-00
def powerpoint_motion_action():
    print("powerpoint_motion_action")
    #################### ▼패키지 임포트▼ ####################
    import cv2
    import pyautogui
    import mediapipe as mp
    import numpy as np
    import time
    import math
    #################### ▲패키지 임포트▲ ####################
    
    #pyautogui.PAUSE = 0.0001    # 딜레이 관련
    #pyautogui.FAILSAFE = False  # 강제 초기화 비활성

    max_num_hands = 1 # 손 갯수

    actions = {0:'middle_Click', 3:'left_click', 1:'Move', 4:'back', 5:'full', 9:'right_click'}

    # MediaPipe hands model
    mp_hands = mp.solutions.hands
    mp_drawing = mp.solutions.drawing_utils
    hands = mp_hands.Hands(
        max_num_hands=max_num_hands,
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5)

    # Gesture recognition model
    file = np.genfromtxt('data/gesture_train.csv', delimiter=',')
    angle = file[:,:-1].astype(np.float32)
    label = file[:, -1].astype(np.float32)
    knn = cv2.ml.KNearest_create()
    knn.train(angle, cv2.ml.ROW_SAMPLE, label)

    cap = cv2.VideoCapture(0)
    start = time.time()

    while True :
        ret, img = cap.read()

        if time.time() - start >= 0.7: # 1초 버튼이 동시에 여러번 눌리는 것을 방지
            start = time.time()
    
            img = cv2.flip(img, 1)
            result = hands.process(img)
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            #img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

            if result.multi_hand_landmarks is not None:
                for res in result.multi_hand_landmarks:
                    joint = np.zeros((21, 3))
                    for j, lm in enumerate(res.landmark):
                        joint[j] = [lm.x, lm.y, lm.z]

                    # Compute angles between joints
                    v1 = joint[[0,1,2,3,0,5,6,7,0,9,10,11,0,13,14,15,0,17,18,19],:] # Parent joint
                    v2 = joint[[1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20],:] # Child joint
                    v = v2 - v1 # [20,3]
                    # Normalize v
                    v = v / np.linalg.norm(v, axis=1)[:, np.newaxis]

                    # Get angle using arcos of dot product
                    angle = np.arccos(np.einsum('nt,nt->n',
                        v[[0,1,2,4,5,6,8,9,10,12,13,14,16,17,18],:], 
                        v[[1,2,3,5,6,7,9,10,11,13,14,15,17,18,19],:])) # [15,]

                    angle = np.degrees(angle) # Convert radian to degree

                    # Inference gesture
                    data = np.array([angle], dtype=np.float32)
                    ret, results, neighbours, dist = knn.findNearest(data, 3)
                    idx = int(results[0][0])

                    # Draw gesture result
                    if idx in actions.keys():
                        cv2.putText(img, text=actions[idx].upper(), 
                                    org=(int(res.landmark[0].x * img.shape[1]), 
                                         int(res.landmark[0].y * img.shape[0] + 20))
                                    , fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, 
                                    color=(255, 255, 255), thickness=2)

                        if idx == 1:
                            print('1')

                        elif idx == 3: 
                            print('3')

                        elif idx == 9:
                            print('9')

                        elif idx == 4:
                            print('4')

                        elif idx == 0:
                            print('0')

                        elif idx == 5:
                            print('5')
                            
                        else:
                            pass

                    # actions = {0:'left_click', 1:'Move', 5:'back', 9:'right_click'}
                    # Other gestures
                    # cv2.putText(img, text=gesture[idx].upper(), org=(int(res.landmark[0].x * img.shape[1]), int(res.landmark[0].y * img.shape[0] + 20)), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255, 255, 255), thickness=2)
                    # mp_drawing.draw_landmarks(img, res, mp_hands.HAND_CONNECTIONS)


            #cv2.imshow('mouse', img)
        if cv2.waitKey(1) == ord('q'):  
            break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
powerpoint_motion_action()

powerpoint_motion_action
5
5
5
5
1
0
0
5
1
1
1
1
0
0
5
0
1
9
5
5
0
5
